# Notebook of ArcPy
This is my learning book of ArcPy.

In [1]:
import arcpy

### 1、图像批量裁剪功能
用一个shp文件对多幅遥感影像进行裁剪，得到多幅影像的同一区域数据。  
输入待裁剪的影像、shp文件和文件格式，输出裁剪完毕的影像。  
主要函数：ExtractByMask_sa

In [ ]:
arcpy.CheckOutExtension("spatial")                      #权限检查
typeall={"TIF":"TIF","img":"img"}

arcpy.env.workspace=arcpy.GetParameterAsText(0)         #定义工作空间,需将待裁剪影像放在一个目录下
Inputfeature=arcpy.GetParameterAsText(1)                #shp文件
OutputFile=arcpy.GetParameterAsText(2)                  #输出文件夹
type1=arcpy.GetParameterAsText(3)                       #格式

type1=typeall[type1]
rasters=arcpy.ListRasters("*",type1)                    #将文件格式为tpye的数据放入rasters中，type:要裁剪的影像格式类型
for raster in rasters:
    arcpy.AddMessage(raster)
    out=OutputFile+"\clip_"+raster                     #这样输出的文件保留了全称和文件格式输出的时候文件名前面加了_clip_
    arcpy.gp.ExtractByMask_sa(raster,Inputfeature,out)
    arcpy.AddMessage(raster+"has done")

### 2、对SHP文件进行分块裁剪
目的：对大范围内的点线数据进行剖分，形成坐标范围较小的局部SHP文件。  
输入：待剖分的管点.shp文件，待剖分的管线.shp文件，输出文件夹；  
输出：基于管线数据进行格网剖分后的格网SHP文件，每个格网文件夹中收纳相匹配的管点和管线SHP文件。  

#### Step.1 定义包括裁剪框在内的相关参数

In [ ]:
# List of coordinates.
coordinates = [
    [4331338, 478876],
    [4333294, 478876],
    [4333294, 481877],
    [4331338, 481877],
    [4331338, 478876]]

# Create an array with a point object for each coordinate pair
array = arcpy.Array([arcpy.Point(y, x) for x, y in coordinates])

# Create a polygon geometry object using the array object
boundary_area = arcpy.Polygon(array)
boundary_line = arcpy.Polyline(array)

# Set the input and output parameters
source = 'C:/Users/Administrator/Desktop/NYJT/能源集团地下管线.gdb/热力管线'
source_point = 'C:/Users/Administrator/Desktop/NYJT/能源集团地下管线.gdb/热力管点'
clip_out = 'C:/Users/Administrator/Desktop/NYJT/Backup/experiment/output.gdb/热力管线_clipped'
clip_out_point = 'C:/Users/Administrator/Desktop/NYJT/Backup/experiment/output.gdb/热力管点_clipped'

#### Step.2 根据裁剪框执行裁剪

In [ ]:
# Use the geometry to clip an input feature class
arcpy.Clip_analysis(source, boundary_area, clip_out)
arcpy.Clip_analysis(source_point, boundary_area, clip_out_point)

#### Step.3 通过相交获取直线被截断处的端点并保存到文件
输出1：裁剪后的SHP文件与Polyline的交点;  
输出2：裁剪后的直线号

In [ ]:
# Read the clip shape file 
rows = arcpy.UpdateCursor(clip_out)  # Update cursor to read the shape file
feature = []                         # Feature array to save the intersect point
pointID = []                          # Line ID array to save the crossed line
count = 0                            # Count the number of intersected points

# Get the intersected point of the polygon area and the shape tube file
for row in rows:
    line = row.getValue('SHAPE')    
    if (not boundary_line.disjoint(line)):                    # adjust whether crossed
        # get the interset point and convert it to PointGeometry
        cross_point = boundary_line.intersect(line, 1)              
        cross_point_geom = arcpy.PointGeometry(cross_point.firstPoint)
        ID = 'TJISM' + str(count).zfill(5)  
        
        # calculate the distance between this point and QSD or ZZD
        QSD_X = row.getValue('QSD_X')
        QSD_Y = row.getValue('QSD_Y')
        ZZD_X = row.getValue('ZZD_X')
        ZZD_Y = row.getValue('ZZD_Y')
        QSD = arcpy.PointGeometry(arcpy.Point(QSD_Y, QSD_X))
        ZZD = arcpy.PointGeometry(arcpy.Point(ZZD_Y, ZZD_X))
        QSD_cross = arcpy.PointGeometry(line.firstPoint)
        ZZD_cross = arcpy.PointGeometry(line.lastPoint)
        if (QSD.distanceTo(QSD_cross) > 1e-5 and QSD.distanceTo(ZZD_cross) > 1e-5):
            row.setValue('起始点号', ID) # QSD have been cut, modify the 起始点号
        if (ZZD.distanceTo(QSD_cross) > 1e-5 and ZZD.distanceTo(ZZD_cross) > 1e-5):
            row.setValue('终止点号', ID) # QSD have been cut, modify the 起始点号            
        rows.updateRow(row)
        
        # add PointGeometry to the output feature
        feature.append(cross_point_geom) 
        pointID.append(ID)
        count = count + 1
        
# Delete cursor and row objects to remove locks on the data
del row
del rows
print("We got %d intersected points." %(count))

#### Step.4 向裁剪后的管点文件中追加数据

In [ ]:
# Create insert cursor for point table
fields = ['SHAPE','物探点号','X','Y','特征','附属物','地面高程','井底高程','井底埋深']
point_rows = arcpy.da.InsertCursor(clip_out_point,fields)
i = 0
for point in feature:
    point_rows.insertRow((point, pointID[i],str(point.firstPoint.Y),str(point.firstPoint.X),'直通点','热力探测点',0,0,0))
    i = i + 1

# Delete cursor and row objects to remove locks on the data
del point_rows
print('New point features has been inserted.')

#### [备份代码]  添加字段及属性信息

In [ ]:
# Output the intersected point
cross_out = 'C:/Users/Administrator/Desktop/NYJT/Backup/experiment/output.gdb/热力管点_added'
arcpy.CopyFeatures_management(feature, cross_out) # 创建shp要素

# Add Field to the element
arcpy.AddField_management(cross_out,'物探点号', 'TEXT')
arcpy.AddField_management(cross_out,'X','TEXT')
arcpy.AddField_management(cross_out,'Y','TEXT')

# Update the attribute tabel
point_rows = arcpy.UpdateCursor(cross_out)
i = 0
for row in point_rows:
    point = row.getValue('SHAPE').firstPoint
    row.setValue('物探点号', pointID[i])
    row.setValue('X', str(point.Y)) # 用repr函数保留更高的精度
    row.setValue('Y', str(point.X)) # str函数可保留12位的精度
    i = i + 1
    point_rows.updateRow(row)
    
# Delete cursor and row objects to remove locks on the data
del row
del point_rows
    
# Print the result
print('Attribute Modified is Finished!')

#### 最终代码，形成一个通用函数

In [2]:
# Cut the point and line shape file and reconnect the start point and end point of the line
# Note: 1.在Line对应的Shape文件中必须手动计算其线段的起始点、终止点坐标，
#         并需以QSD_X,QSD_Y,ZZD_X,ZZD_Y的形式进行存储；
#       2.在管点文件中必须包含'SHAPE'、'物探点号'、'X'、'Y'、'特征'、'附属物'、
#         '地面高程'、'井底高程'、'井底埋深'等9个字段，字段不可缺省。
def CutShapeByArray(Array, GDBName, LineSHP, PointSHP, Output):
    
    # Create a polygon geometry object using the array object
    boundary_area = arcpy.Polygon(array)
    boundary_line = arcpy.Polyline(array)

    # Set the input and output parameters
    arcpy.CreateFileGDB_management(Output, GDBName)
    line_clip_out = Output  + '/' + GDBName + '/热力管线'
    point_clip_out = Output + '/' + GDBName + '/热力管点'
    
    # Clip the line and the point
    arcpy.Clip_analysis(LineSHP, boundary_area, line_clip_out)
    arcpy.Clip_analysis(PointSHP, boundary_area, point_clip_out)
    
    # Read the clipped shape file of line
    line_rows = arcpy.UpdateCursor(line_clip_out)  # Update cursor to read the shape file
    feature = []                                   # Feature array to save the intersect point
    pointID = []                                   # Line ID array to save the crossed line
    count = 0                                      # Count the number of intersected points
    have_crossed_feature = False                  # Try to make sure whether there is crossed feature

    # Get the intersected point of the polygon area and the shape tube file
    for row in line_rows:
        line = row.getValue('SHAPE')
        if (not boundary_line.disjoint(line)): # adjust whether crossed
            
            # get the interset point and convert it to PointGeometry
            cross_point = boundary_line.intersect(line, 1)              
            cross_point_geom = arcpy.PointGeometry(cross_point.firstPoint)
            ID = 'TJISM' + str(count).zfill(5)  

            # calculate the distance between this point and QSD or ZZD
            QSD_X = row.getValue('QSD_X')
            QSD_Y = row.getValue('QSD_Y')
            ZZD_X = row.getValue('ZZD_X')
            ZZD_Y = row.getValue('ZZD_Y')
            QSD = arcpy.PointGeometry(arcpy.Point(QSD_Y, QSD_X))
            ZZD = arcpy.PointGeometry(arcpy.Point(ZZD_Y, ZZD_X))
            QSD_cross = arcpy.PointGeometry(line.firstPoint)
            ZZD_cross = arcpy.PointGeometry(line.lastPoint)
            if (QSD.distanceTo(QSD_cross) > 1e-5 and QSD.distanceTo(ZZD_cross) > 1e-5):
                row.setValue('起始点号', ID) # QSD have been cut, modify the 起始点号
            if (ZZD.distanceTo(QSD_cross) > 1e-5 and ZZD.distanceTo(ZZD_cross) > 1e-5):
                row.setValue('终止点号', ID) # QSD have been cut, modify the 起始点号 
                
            # Update the row of the line rows
            line_rows.updateRow(row)

            # add PointGeometry to the output feature
            feature.append(cross_point_geom) 
            pointID.append(ID)
            count = count + 1
        have_crossed_feature = True
            
    if(not count == 0):
        # Create insert cursor for point table
        fields = ['SHAPE','物探点号','X','Y','特征','附属物','地面高程','井底高程','井底埋深']
        point_rows = arcpy.da.InsertCursor(point_clip_out, fields)
        i = 0
        for point in feature:
            point_rows.insertRow(
                (point, pointID[i],str(point.firstPoint.Y),str(point.firstPoint.X),
                 '直通点','热力探测点',0,0,0))
            i = i + 1

        # Delete cursor and row objects to remove locks on the data
        del line_rows
        del point_rows
        
    else:
        if (have_crossed_feature):
            with open(Output + '/Valued.txt','a+') as file:
                file.write(GDBName + '\n')
    
    # Print the result
    print('Shape File Cut for '+ GDBName + ' is Finished!')

### 3、计算包络矩形并由外围包括矩形计算地理分块
理想的操作是自适应化的地理分块算法，使得：  
[1]. 地理分块尽可能小；  
[2]. 地理分块避免横跨直线特征密集的区域；  
[3]. 地理分块时要避免直线恰好穿过某一点的情况；  
[4]. 地理分块时要尽可能形成要素簇以减少不必要的分割；  
[5]. 减小对空白区域的分块操作。

**解决方案<2020-09-04>**  
首先采取常规方式，计算图像包络矩形，然后构建均匀网格进行分割。

In [3]:
# Get the extent coordinates of the shape file
def GetExtent(Shape):
    cursor = arcpy.da.SearchCursor(Shape, ['SHAPE@XY'])
    XMin = 9999999999.0
    YMin = 9999999999.0
    XMax = -9999999999.0
    YMax = -9999999999.0
    for row in cursor:
        x, y = row[0]
        if(x < XMin):
            XMin = x
        if(y < YMin):
            YMin = y
        if(x > XMax):
            XMax = x
        if(y > YMax):
            YMax = y
    return arcpy.Extent(XMin - 1, YMin - 1, XMax + 1, YMax + 1)

In [ ]:
# TOOLs: Calculate the cut area grid and output it to the file
fc = 'C:/Users/Administrator/Desktop/NYJT/能源集团地下管线.gdb/热力管点'
extent = GetExtent(fc)
step = 10
grid_width = (extent.XMax - extent.XMin) / step
grid_height = (extent.YMax - extent.YMin) / step
feature = []
cross_out = 'C:/Users/Administrator/Desktop/NYJT/Backup/experiment/output.gdb/热力管点_extent'
for i in range(0, step):
    for j in range(0, step):
        point_1 = arcpy.Point(extent.lowerLeft.X + i * grid_width, extent.lowerLeft.Y + j * grid_height)
        point_2 = arcpy.Point(extent.lowerLeft.X + (i + 1) * grid_width, extent.lowerLeft.Y + j * grid_height)
        point_3 = arcpy.Point(extent.lowerLeft.X + (i + 1) * grid_width, extent.lowerLeft.Y + (j + 1) * grid_height)
        point_4 = arcpy.Point(extent.lowerLeft.X + i * grid_width, extent.lowerLeft.Y + (j + 1) * grid_height)
        array = arcpy.Array([point_1, point_2, point_3, point_4])
        rectangle = arcpy.Polygon(array)
        feature.append(rectangle)
arcpy.CopyFeatures_management(feature, cross_out) 

In [4]:
# Set the line shp and point shp
line_shp = 'C:/Users/Administrator/Desktop/NYJT/能源集团地下管线.gdb/热力管线'
point_shp = 'C:/Users/Administrator/Desktop/NYJT/能源集团地下管线.gdb/热力管点'
output_folder = 'C:/Users/Administrator/Desktop/experiment/test3'

# Calculate the extent information of the point shp
extent = GetExtent(point_shp)
step = 10
grid_width = (extent.XMax - extent.XMin) / step
grid_height = (extent.YMax - extent.YMin) / step

# Iterate the cut grid shape file
for i in range(0, step):
    for j in range(0, step):
        point_1 = arcpy.Point(extent.lowerLeft.X + i * grid_width, extent.lowerLeft.Y + j * grid_height)
        point_2 = arcpy.Point(extent.lowerLeft.X + (i + 1) * grid_width, extent.lowerLeft.Y + j * grid_height)
        point_3 = arcpy.Point(extent.lowerLeft.X + (i + 1) * grid_width, extent.lowerLeft.Y + (j + 1) * grid_height)
        point_4 = arcpy.Point(extent.lowerLeft.X + i * grid_width, extent.lowerLeft.Y + (j + 1) * grid_height)
        array = arcpy.Array([point_1, point_2, point_3, point_4, point_1])
        gdb_name = 'GDB' + '_' + str(i).zfill(2) + '_' + str(j).zfill(2) + '.gdb'
        CutShapeByArray(array, gdb_name, line_shp, point_shp, output_folder)
        
print('Finshed!')

Shape File Cut for GDB_00_00.gdb is Finished!
Shape File Cut for GDB_00_01.gdb is Finished!
Shape File Cut for GDB_00_02.gdb is Finished!
Shape File Cut for GDB_00_03.gdb is Finished!
Shape File Cut for GDB_00_04.gdb is Finished!
Shape File Cut for GDB_00_05.gdb is Finished!
Shape File Cut for GDB_00_06.gdb is Finished!
Shape File Cut for GDB_00_07.gdb is Finished!
Shape File Cut for GDB_00_08.gdb is Finished!
Shape File Cut for GDB_00_09.gdb is Finished!
Shape File Cut for GDB_01_00.gdb is Finished!
Shape File Cut for GDB_01_01.gdb is Finished!
Shape File Cut for GDB_01_02.gdb is Finished!
Shape File Cut for GDB_01_03.gdb is Finished!
Shape File Cut for GDB_01_04.gdb is Finished!
Shape File Cut for GDB_01_05.gdb is Finished!
Shape File Cut for GDB_01_06.gdb is Finished!
Shape File Cut for GDB_01_07.gdb is Finished!


SystemError: <built-in function getattr> returned NULL without setting an error